In [1]:
import sys
sys.path.append('../carbs/src/')

import carbon
import scheduling as gaia_sched
import power_consumption_profiles as pcp
import plotly.express as px
import plotly.graph_objects as go
from scheduling.suspend_phases_scheduling_policy import SchedulerDebugOptions
from typing import List
import pulp


In [8]:
start_index = 500

carbon_model = carbon.get_carbon_model("DE-hourly-start-july", start_index, "ORACLE", True).reindex(0)

print(carbon_model.df)

df = carbon_model.df.head(80)

fig_carbon = px.line(df, x=df.index, y="carbon_intensity_avg")

scheduler = gaia_sched.SuspendSchedulingDynamicPowerPolicy(None, carbon_model)

vincent_stawp = pcp.PowerFunction({
    'startup': [
        {'name': 'Wake up', 'duration': 4, 'power': 20}, # lol
        {'name': 'Coffee', 'duration': 2, 'power': 30}
    ],
    'work': [
        {'name': 'Work on Thesis', 'duration': 4, 'power': 20},
        {'name': 'Mensa', 'duration': 2, 'power': 1000},
        {'name': 'Get Feedback', 'duration': 2, 'power': 20},
        {'name': 'Work on Thesis', 'duration': 4, 'power': 20},
    ]
})

options = SchedulerDebugOptions(
        use_startup=True, 
        dynamic_power=True, 
        use_progress=True, 
        linearize=True,
        timelimit=None,
        scale_time=False,
)

result_1 = scheduler.find_execution_times(carbon_model, 30, vincent_stawp, options)

def find_upper_lower_index_for_phases(bool_list: List[bool]):
    groups = []
    start_index = None

    for i, value in enumerate(bool_list):
        if value: 
            if start_index is None:
                start_index = i
        else:
            if start_index is not None:
                groups.append((start_index, i - 1)) 
                start_index = None

    if start_index is not None:
        groups.append((start_index, len(bool_list) - 1))

    # Output the result
    return groups

phases_variables_with_name = [
    *[(value['variable'], key) for (key, value) in result_1["lin_function_dicts"]['startup'].items()],
    *[(value['variable'], key) for (key, value) in result_1["lin_function_dicts"]['work'].items()]
]

for variable, name in phases_variables_with_name:
    bools = [True if pulp.value(variable[t]) > 0 else False for t in range(len(variable))]
    groups = find_upper_lower_index_for_phases(bools)
    for start, end in groups:
        print(f"{start}, {end}, {name}")
        fig_carbon.add_shape(type='rect',
            name=name, layer="below",
            label=dict(text=name, textangle=-90),
            x0=start, y0=0, x1=end+1, y1=0.4,
            line=dict(color='rgba(0, 0, 0, 0)'),
            row=1, col=1,
            fillcolor="red"
        )

fig_carbon.show()

      index  carbon_intensity_avg                  datetime
0       500                 0.426  2024-07-21T20:00:00.000Z
1       501                 0.416  2024-07-21T21:00:00.000Z
2       502                 0.412  2024-07-21T22:00:00.000Z
3       503                 0.416  2024-07-21T23:00:00.000Z
4       504                 0.407  2024-07-22T00:00:00.000Z
...     ...                   ...                       ...
1435   1935                 0.230  2024-09-19T15:00:00.000Z
1436   1936                 0.513  2024-09-23T16:00:00.000Z
1437   1937                 0.521  2024-09-23T17:00:00.000Z
1438   1938                 0.525  2024-09-23T18:00:00.000Z
1439   1939                 0.525  2024-09-23T19:00:00.000Z

[1440 rows x 3 columns]
scaled deadlines is: 30, spt: 1
Set parameter Username
Set parameter LogFile to value "gurobi.log"
Set parameter Threads to value 4
Using license file /opt/gurobi1103/gurobi.lic
Academic license - for non-commercial use only - expires 2025-08-12

Gurobi O